In [1]:
import numpy as np

# Part 1

In [2]:
test = ['class: 1-3 or 5-7',
        'row: 6-11 or 33-44',
        'seat: 13-40 or 45-50',
        '',
        'your ticket:',
        '7,1,14',
        '',
        'nearby tickets:',
        '7,3,47',
        '40,4,50',
        '55,2,20',
        '38,6,12']

In [3]:
data = np.genfromtxt('day16_input.txt', delimiter='\n', dtype=str)

In [4]:
def get_ticket_data(data):
    rules = {}
    my_ticket = []
    nr_ticket = []
    typ = 'rule'
    
    for i in range(0, len(data)):
        line = data[i]
        if 'or' in line:
            split = line.split(' ')
            if len(split) == 4:
                key = split[0][:-1]
            elif len(split) == 5:
                key = split[0]
                key += ' '+split[1][:-1]
            
            low = split[-3].split('-')
            low[0] = int(low[0])
            low[1] = int(low[1])
            
            hig = split[-1].split('-')
            hig[0] = int(hig[0])
            hig[1] = int(hig[1])
            
            rules[key] = [low, hig]
            
        elif line == 'your ticket:':
            i += 1
            line = data[i]
            my_ticket = line.split(',')
            for j in range(0, len(my_ticket)):
                my_ticket[j] = int(my_ticket[j])
                
        elif line == 'nearby tickets:':
            while True:
                i += 1
                if i >= len(data):
                    break
                line = data[i]
                ticket = line.split(',')
                for j in range(0, len(ticket)):
                    ticket[j] = int(ticket[j])
                nr_ticket.append(ticket)
            break
            
    return rules, my_ticket, nr_ticket

def get_invalid_values(data):
    rules, my_ticket, nr_ticket = get_ticket_data(data)
    
    bad = []
    remove = []
    for h in range(0, len(nr_ticket)):
        ticket = nr_ticket[h]
        for i in range(0, len(ticket)):
            bad_rules = 0
            for key in rules.keys():
                if ticket[i] < rules[key][0][0] or ticket[i] > rules[key][1][1] or\
                    (ticket[i] > rules[key][0][1] and ticket[i] < rules[key][1][0]):
                    bad_rules += 1
            if bad_rules == len(rules.keys()):
                bad.append(ticket[i])
                remove.append(h)
                
    nr_ticket = np.delete(np.array(nr_ticket), remove, axis=0).tolist()
                    
    return np.sum(bad), rules, my_ticket, nr_ticket

print(get_invalid_values(test)[0])

71


In [5]:
print('Part 1 Result:', get_invalid_values(data)[0])

Part 1 Result: 23036


# Part 2

In [6]:
test1 = ['class: 0-1 or 4-19',
         'row: 0-5 or 8-19',
         'seat: 0-13 or 16-19',
         '',
         'your ticket:',
         '11,12,13',
         '',
         'nearby tickets:',
         '3,9,18',
         '15,1,5',
         '5,14,9']

In [7]:
def get_class_names(data):
    _, rules, my_ticket, nr_tickets = get_invalid_values(data)
    
    nr_tickets.append(my_ticket)
    tickets = np.array(nr_tickets)
    
    possible = {}
    for i in range(0, len(tickets[0])):
        for key in rules.keys():
            bad_l = np.where(tickets[:,i] < rules[key][0][0])[0]
            bad_u = np.where(tickets[:,i] > rules[key][1][1])[0]
            bad = np.hstack((bad_l, bad_u))
            
            bad_ml = np.where(tickets[:,i] > rules[key][0][1])[0]
            bad_mu = np.where(tickets[:,i] < rules[key][1][0])[0]
            bad_m = np.intersect1d(bad_ml, bad_mu)
            bad = np.hstack((bad, bad_m))
            
            if len(bad) == 0:
                if i not in possible.keys():
                    possible[i] = [key]
                else:
                    possible[i].append(key)
                    
    real = {}
    while len(real.keys()) < len(rules.keys()):
        for key in possible.keys():
            if len(possible[key]) == 1:
                real[key] = possible[key][0]
                
        for key in real.keys():
            for p_key in possible.keys():
                if real[key] in possible[p_key]:
                    possible[p_key].remove(real[key])
                
    return real, my_ticket

def get_part_2(data):
    real, my_ticket = get_class_names(data)
    
    result = 1
    for i in range(0, len(my_ticket)):
        if 'departure' in real[i]:
            result *= my_ticket[i]
            
    return result
    
print(get_class_names(test1)[0])

{0: 'row', 1: 'class', 2: 'seat'}


In [8]:
print('Part 2 Result:', get_part_2(data))

Part 2 Result: 1909224687553
